# 03-2. 타이타닉 데이터 로지스틱 회귀

<br><hr>

## ✅로지스틱 회귀분석의 활용

#### *타이타닉 탑승자 생존여부 예측*

## 00. 기본 설정

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 메시지는 출력되지 않게 합니다.
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 주피터 노트북에서 그래프가 보이는 설정
%matplotlib notebook

In [3]:
# 한글 font 설정

import platform
import matplotlib.font_manager as fm

#matplotlib 패키지 한글 깨짐 처리 시작
#------------------------------------------------------------------------------------
# 운영체제별 한글 폰트 설정

if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
    
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

In [4]:
# 글씨 선명하게 출력하는 설정

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

<hr>

## 01. 데이터 불러오기

In [5]:
# seaborn에서 제공하는 예시 데이터 (titanic 불러오기)
titanic = sns.load_dataset('titanic')
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [6]:
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

In [7]:
# 문자형 변수들을 숫자형으로 변경
encoder = LabelEncoder()
encoder.fit(titanic['sex'])
sex = encoder.transform(titanic['sex'])

In [8]:
titanic['sex'] = sex
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,1,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,0,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,0,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,0,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,1,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [9]:
model = sm.Logit.from_formula('survived ~ pclass + sex + age + fare + parch + sibsp', data=titanic)
# 모델 학습
logit = model.fit()

Optimization terminated successfully.
         Current function value: 0.445244
         Iterations 6


In [10]:
print(logit.summary())

                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      707
Method:                           MLE   Df Model:                            6
Date:                Wed, 20 Nov 2024   Pseudo R-squ.:                  0.3408
Time:                        11:51:17   Log-Likelihood:                -317.90
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 5.727e-68
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.3890      0.604      8.926      0.000       4.206       6.572
pclass        -1.2422      0.163     -7.612      0.000      -1.562      -0.922
sex           -2.6348      0.220    -11.998      0.0

In [11]:
# 각각의 독립변수의 회귀계수 유의성 확인
logit.pvalues  # fare/parch 변수는 유의하지 않음

Intercept    4.411929e-19
pclass       2.693967e-14
sex          3.645052e-33
age          7.703970e-08
fare         3.862666e-01
parch        6.143580e-01
sibsp        3.174516e-03
dtype: float64

In [14]:
# 각각의 독립변수의 회귀계수 확인
logit.params
# (pclass/sex/age/sibsp 모두 음수이므로 독립변수의 값이 증가할 때 생존 가능성이 낮아진다고 볼 수 있음)

Intercept    5.389003
pclass      -1.242249
sex         -2.634845
age         -0.043953
fare         0.002160
parch       -0.061937
sibsp       -0.375755
dtype: float64

In [15]:
# 오즈비
np.exp(logit.params)
# age의 오즈비가 가장 크고, sex의 오즈비가 가장 작음
# age 값이 증가할 때 생존하지 않을 가능성이 생존할 가능성의 0.957배
# 성별에 따른 오즈비: 남성(sex=1)의 생존 비율이 여성(sex=0)의 생존 비율의 0.072배

Intercept    218.984972
pclass         0.288734
sex            0.071730
age            0.956999
fare           1.002162
parch          0.939942
sibsp          0.686771
dtype: float64